In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

from copy import copy
import datetime
import pickle
from geopy.geocoders import Nominatim

In [2]:
filename = "xgb_routeOptimization.sav"

In [3]:
loaded_model = pickle.load(open(filename, 'rb'))

[15:18:37] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.


In [13]:
#Sample date

date_list = [4, 6, 2019] #April 6, 2016

year = int(date_list[2])
month = int(date_list[1])
day = int(date_list[0])

my_date = datetime.date(year, month, day)

In [4]:
#Sample test locations
test_locations = {'current_location':(40.7651519775,-73.9767456055),
                  'L1': (40.764198, -73.910785),
                  'L2': (40.815421, -73.941761),
                  'L3': (40.819688, -73.915091),
                  'L4': (40.768790, -73.953285),
                  'L5': (40.734851, -73.952950),
                  'L6': (40.743613, -73.977998),
                  'L7': (40.745313, -73.993793),
                  'L8': (40.662713, -73.946101),
                  'L9': (40.703761, -73.886496),
                  'L10': (40.713620, -73.943076),
                  'L11': (40.725212, -73.809179)
             }

In [5]:
geolocator = Nominatim()
addresses = []

for key in test_locations:
    location = geolocator.reverse(test_locations[key])
    addresses.append(location.address)

/home/india/anaconda2/envs/pytorch_learning/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [6]:
addresses


['1421, 6th Avenue, Midtown, Manhattan Community Board 5, Manhattan, New York County, New York, 10019, United States of America',
 '43-11, 28th Avenue, Steinway, Queens, Queens County, New York, 11103, United States of America',
 '137, West 136th Street, Harlem, Manhattan Community Board 10, Manhattan, New York County, New York, 10030, United States of America',
 '424, East 155th Street, Melrose, The Bronx, Bronx County, New York, 10455, United States of America',
 '435, East 74th Street, Upper East Side, Manhattan Community Board 8, Manhattan, New York County, New York, 10021, United States of America',
 '211, Freeman Street, Greenpoint, Brooklyn, Kings County, New York, 11222, United States of America',
 '232, East 32nd Street, Kips Bay, Manhattan Community Board 5, Manhattan, New York County, New York, 10016, United States of America',
 '159, West 25th Street, Penn South, Chelsea, Manhattan Community Board 4, Manhattan, New York County, New York, 10001, United States of America',
 '

In [7]:
test_addresses = {'L1': '424 East 155th Street NY',
                  'L2': '137 West 136th Street NY',
                  'L3': '43-11 28th Avenue NY',
                  'L4': '435 East 74th Street NY',
                  'L5': '211 Freeman Street NY',
                  'L6': '232 East 32nd Street NY',
                  'L7': '159 West 25th Street NY',
                  'L8': '486 Brooklyn Avenue NY',
                  'L9': '70-38 67th Place NY',
                  'L10': '194 Devoe Street NY',
                  'L11': '158-46 76th Avenue NY'
             }

In [8]:
def create_guess(points):
    """
    Creates a possible path between all points, returning to the original.
    Input: List of point IDs
    """
    guess = copy(points)
    np.random.shuffle(guess)
    guess.remove("current_location")
    #guess.append(guess[0])
    guess.insert(0, "current_location")
    return list(guess)

create_guess(list(test_locations.keys()))

['current_location',
 'L2',
 'L5',
 'L3',
 'L6',
 'L11',
 'L8',
 'L10',
 'L9',
 'L4',
 'L7',
 'L1']

In [9]:
def create_generation(points, population=100):
    """
    Makes a list of guessed point orders given a list of point IDs.
    Input:
    points: list of point ids
    population: how many guesses to make
    """
    generation = [create_guess(points) for _ in range(population)]
    return generation

test_generation = create_generation(list(test_locations.keys()), population=10)
print(test_generation)

[['current_location', 'L9', 'L1', 'L10', 'L5', 'L6', 'L7', 'L11', 'L4', 'L2', 'L3', 'L8'], ['current_location', 'L6', 'L10', 'L4', 'L3', 'L11', 'L1', 'L2', 'L9', 'L5', 'L7', 'L8'], ['current_location', 'L7', 'L11', 'L4', 'L3', 'L8', 'L1', 'L5', 'L9', 'L6', 'L10', 'L2'], ['current_location', 'L6', 'L9', 'L10', 'L11', 'L8', 'L3', 'L2', 'L1', 'L5', 'L7', 'L4'], ['current_location', 'L10', 'L3', 'L2', 'L4', 'L9', 'L5', 'L8', 'L1', 'L7', 'L6', 'L11'], ['current_location', 'L2', 'L10', 'L3', 'L1', 'L7', 'L11', 'L5', 'L8', 'L6', 'L4', 'L9'], ['current_location', 'L5', 'L7', 'L2', 'L1', 'L10', 'L4', 'L6', 'L3', 'L8', 'L11', 'L9'], ['current_location', 'L1', 'L8', 'L3', 'L6', 'L4', 'L5', 'L7', 'L9', 'L2', 'L10', 'L11'], ['current_location', 'L8', 'L10', 'L1', 'L6', 'L11', 'L7', 'L3', 'L9', 'L4', 'L2', 'L5'], ['current_location', 'L8', 'L4', 'L11', 'L3', 'L2', 'L1', 'L9', 'L7', 'L5', 'L6', 'L10']]


In [10]:
def travel_time_between_points(point1_id, point2_id, hour, date,pickup_minute = 0):
    """
    Given two points, this calculates travel between them based on a XGBoost predictive model
    """
    
    model_data = {'pickup_longitude' : point1_id[1],
                  'pickup_latitude' : point1_id[0],
                  'dropoff_longitude' : point2_id[1],
                  'dropoff_latitude' : point2_id[0],
                  'latitude_difference' : point2_id[0] - point1_id[0],
                  'longitude_difference' : point2_id[1] - point1_id[1],
                  'distance' : 0.621371 * 6371 * (abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(point2_id[0] - point1_id[0]) * np.pi / 180) / 2))), 
                                  np.sqrt(1-(np.square(np.sin((abs(point2_id[0] - point1_id[0]) * np.pi / 180) / 2)))))) + \
                                     abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(point2_id[1] - point1_id[1]) * np.pi / 180) / 2))), 
                                  np.sqrt(1-(np.square(np.sin((abs(point2_id[1] - point1_id[1]) * np.pi / 180) / 2)))))))
                 }

    df = pd.DataFrame([model_data], columns=model_data.keys())
    
    pred = np.exp(loaded_model.predict(xgb.DMatrix(df))) - 1
    
    return pred[0]

In [11]:
coordinates = test_locations
#coordinates['current_location']=current_location
#print (coordinates["current_location"])

In [14]:
def fitness_score(guess):
    """
    Loops through the points in the guesses order and calculates
    how much distance the path would take to complete a loop.
    Lower is better.
    """
    score = 0
    for ix, point_id in enumerate(guess[:-1]):
        score += travel_time_between_points(coordinates[point_id], coordinates[guess[ix+1]], 11, my_date)
    return score

def check_fitness(guesses):
    """
    Goes through every guess and calculates the fitness score. 
    Returns a list of tuples: (guess, fitness_score)
    """
    fitness_indicator = []
    for guess in guesses:
        fitness_indicator.append((guess, fitness_score(guess)))
    return fitness_indicator

print(check_fitness(test_generation))

[(['current_location', 'L9', 'L1', 'L10', 'L5', 'L6', 'L7', 'L11', 'L4', 'L2', 'L3', 'L8'], 270.81771755218506), (['current_location', 'L6', 'L10', 'L4', 'L3', 'L11', 'L1', 'L2', 'L9', 'L5', 'L7', 'L8'], 306.498104095459), (['current_location', 'L7', 'L11', 'L4', 'L3', 'L8', 'L1', 'L5', 'L9', 'L6', 'L10', 'L2'], 335.36106300354004), (['current_location', 'L6', 'L9', 'L10', 'L11', 'L8', 'L3', 'L2', 'L1', 'L5', 'L7', 'L4'], 293.9102592468262), (['current_location', 'L10', 'L3', 'L2', 'L4', 'L9', 'L5', 'L8', 'L1', 'L7', 'L6', 'L11'], 286.4690990447998), (['current_location', 'L2', 'L10', 'L3', 'L1', 'L7', 'L11', 'L5', 'L8', 'L6', 'L4', 'L9'], 295.2116508483887), (['current_location', 'L5', 'L7', 'L2', 'L1', 'L10', 'L4', 'L6', 'L3', 'L8', 'L11', 'L9'], 309.91514778137207), (['current_location', 'L1', 'L8', 'L3', 'L6', 'L4', 'L5', 'L7', 'L9', 'L2', 'L10', 'L11'], 308.6469078063965), (['current_location', 'L8', 'L10', 'L1', 'L6', 'L11', 'L7', 'L3', 'L9', 'L4', 'L2', 'L5'], 334.1544876098633)

In [15]:
def get_breeders_from_generation(guesses, take_best_N=10, verbose=False):
    """
    This sets up the breeding group for the next generation. You have
    to be very careful how many breeders you take, otherwise your
    population can explode. These two, plus the "number of children per couple"
    in the make_children function must be tuned to avoid exponential growth or decline!
    """
    # First, get the top guesses from last time
    fit_scores = check_fitness(guesses)
    sorted_guesses = sorted(fit_scores, key=lambda x: x[1]) # sorts so lowest is first, which we want
    new_generation = [x[0] for x in sorted_guesses[:take_best_N]]
    best_guess = new_generation[0]
    
    if verbose:
        # If we want to see what the best current guess is!
        print(best_guess)
    return best_guess

In [15]:
"""def evolve_to_solve(current_generation,max_generations, take_best_N, take_random_N,
                    mutation_rate, print_every_n_generations, verbose=False):"""
    """
    Takes in a generation of guesses then evolves them over time using our breeding rules.
    Continue this for "max_generations" times.
    Inputs:
    current_generation: The first generation of guesses
    max_generations: how many generations to complete
    take_best_N: how many of the top performers get selected to breed
    take_random_N: how many random guesses get brought in to keep genetic diversity
    mutation_rate: How often to mutate (currently unused)
    children_per_couple: how many children per breeding pair
    print_every_n_geneartions: how often to print in verbose mode
    verbose: Show printouts of progress
    Returns:
    fitness_tracking: a list of the fitness score at each generations
    best_guess: the best_guess at the end of evolution
    """
    """fitness_tracking = []
    for i in range(max_generations):
        if verbose and not i % print_every_n_generations and i > 0:
            print("Generation %i: "%i, end='')
            print(len(current_generation))
            print("Current Best Score: ", fitness_tracking[-1])
            is_verbose = True
        else:
            is_verbose = False
        best_guess = get_breeders_from_generation(current_generation, 
                                                            take_best_N=take_best_N, take_random_N=take_random_N, 
                                                            verbose=is_verbose, mutation_rate=mutation_rate)
        fitness_tracking.append(fitness_score(best_guess))
        #current_generation = make_children(breeders, children_per_couple=children_per_couple)
    
    return fitness_tracking, best_guess

current_generation = create_generation(list(test_locations.keys()),population=500)
fitness_tracking, best_guess = evolve_to_solve(current_generation, 30, 150, 70, 0.5, 5, verbose=True)"""

IndentationError: unexpected indent (<ipython-input-15-5091cdf58749>, line 3)

In [16]:
def evolve_to_solve(current_generation,take_best_N, verbose=False):
    fitness_tracking = []
    
    best_guess = get_breeders_from_generation(current_generation, take_best_N=take_best_N, verbose=True)
    fitness_tracking.append(fitness_score(best_guess))
    print("Current Best Score: ", fitness_tracking[-1])
    return fitness_tracking, best_guess

current_generation = create_generation(list(test_locations.keys()),population=5)


fitness_tracking, best_guess = evolve_to_solve(current_generation, 150,verbose=True)

['current_location', 'L8', 'L7', 'L3', 'L2', 'L1', 'L9', 'L10', 'L6', 'L5', 'L4', 'L11']
Current Best Score:  289.16800689697266
